In [1]:
#We install all the required libraries
!pip install flask
!pip install flask_restful
!pip install pandas
!pip install yfinance
!pip install pmdarima
!pip install flask-cors

In [ ]:
#We import the libraries
import yfinance as yf
from pmdarima import auto_arima 
import pandas as pd
from pandas.plotting import lag_plot
from sklearn import preprocessing
import warnings
warnings.filterwarnings('ignore')
from statsmodels.tsa.arima.model import ARIMA
from flask_restful import Resource, Api
from flask import Flask
from flask import request, jsonify
from flask_cors import CORS, cross_origin
import time, datetime
import json
import matplotlib.pyplot as plt
import random

#We create empty dataframe to stores predictions 
data_frame_predicted = pd.DataFrame()
data_frame_predicted["x"] = []
data_frame_predicted["y"] = []


#We create the web service 
app = Flask(__name__)
cors = CORS(app)
app.config['CORS_HEADERS'] = 'Content-Type'

@cross_origin()
@app.route('/', methods=['GET'])
def getPredict():
    global data_frame_predicted #to be able to modify global variable (allows to store previous predictions and update with new ones)
    
    #Date format
    date_format = "%H:%M - %d/%m/%y"
    
    #We get the dataframe with stock prices  
    df = yf.download(tickers="AMZN", period="15d", interval="2m") #Amazon prices for the last 15 days every 2 minutes
    df = df.drop(columns=["Open",	"High",	"Low",	"Adj Close",	"Volume"]) #We only keep Close price
    x = df.values #returns a numpy array of closing prices
    
    min_max_scaler = preprocessing.MinMaxScaler() 
    x_scaled = min_max_scaler.fit_transform(x) #We scale the data from 0 to 1
    df["Close"] = x_scaled #replaces closing prices with scaled ones
    
    
    #Train data
    train_data = df #We train the model with the last 15 days prices
    
    model_autoARIMA = auto_arima(train_data, start_p=1, start_q=1,
                          test='adf',       # use adftest to find optimal 'd'
                          max_p=7, max_q=7, # maximum p and q
                          m=262800,              # frequency of series (every 2 min)
                          d=None,           # let model determine 'd'
                          seasonal=False,   # No Seasonality
                          start_P=0, 
                          D=0, 
                          trace=True,
                          error_action='ignore',  
                          suppress_warnings=True, 
                          stepwise=True,
                          n_jobs=-1,
                          return_valid_fits = False)

    #Prediction 
    prediction_array = model_autoARIMA.predict(n_periods=5) #predicts next 5 observations (next 10 min)
    print(prediction_array)
    
    
    #Index list (for plotting)
    dates_list = df[-100:].index.tolist() #last 3h and half dates approx
    
    #Storing most recent actual prices 
    current_time_series = [] #empty array that will store most recent data to plot
    for date_index in dates_list: 
        current_time_series.append({"x": date_index.strftime(date_format), "y":df.loc[date_index, "Close"]}) #dictionary x:dates, y:prices
    
    
    #Replacing obsolete predictions with newest predictions and adding new one
    
    current_time = df.index.tolist()[-1:][0] #we take the last available date 

    for pred in prediction_array:
        current_time = current_time + datetime.timedelta(minutes = 2) #identifies dates of the predictions 
        if current_time.strftime(date_format) in data_frame_predicted["x"].tolist():
            data_frame_predicted.loc[data_frame_predicted["x"] == current_time.strftime(date_format), 'y'] = pred #Store predicted value when the corresponding actual value happens
        else: 
            time_and_predict_json = {"x":current_time.strftime(date_format), "y":pred}            
            data_frame_predicted = data_frame_predicted.append(time_and_predict_json, ignore_index = True) #Adding new predictions to data_frame_predicted
            
            
    #Creating list with dates of the predictions        
    dates_list = data_frame_predicted["x"].tolist() 
    
    #Creating array with all previous and current predicted data
    prediction_data = []
    for date_index in dates_list: 
        prediction_data.append({"x": date_index, "y":data_frame_predicted[data_frame_predicted["x"] == date_index]["y"].tolist()[0]}) #dictionary x:dates, y:predicted price of corresponding date
    
        
    #We create the JSON object containing the predictions and actual data (last 3h and half)    
    jsonObject = {"current_data":current_time_series,"prediction_data":prediction_data}
    return json.dumps(jsonObject)
    
app.run()

0.40401663494444073
 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
127.0.0.1 - - [05/Jun/2022 11:32:59] "OPTIONS / HTTP/1.1" 200 -


[*********************100%***********************]  1 of 1 completed
Performing stepwise search to minimize aic
 ARIMA(1,1,1)(0,0,0)[0] intercept   : AIC=-18195.765, Time=0.37 sec
 ARIMA(0,1,0)(0,0,0)[0] intercept   : AIC=-18199.610, Time=0.27 sec
 ARIMA(1,1,0)(0,0,0)[0] intercept   : AIC=-18197.667, Time=0.46 sec


127.0.0.1 - - [05/Jun/2022 11:33:01] "GET / HTTP/1.1" 200 -


 ARIMA(0,1,1)(0,0,0)[0] intercept   : AIC=-18197.666, Time=0.73 sec
 ARIMA(0,1,0)(0,0,0)[0]             : AIC=-18200.680, Time=0.11 sec

Best model:  ARIMA(0,1,0)(0,0,0)[0]          
Total fit time: 1.970 seconds
[0.8676366 0.8676366 0.8676366 0.8676366 0.8676366]
[*********************100%***********************]  1 of 1 completed
Performing stepwise search to minimize aic
 ARIMA(1,1,1)(0,0,0)[0] intercept   : AIC=-18195.765, Time=0.46 sec
 ARIMA(0,1,0)(0,0,0)[0] intercept   : AIC=-18199.610, Time=0.31 sec
 ARIMA(1,1,0)(0,0,0)[0] intercept   : AIC=-18197.667, Time=0.37 sec


...

 ARIMA(0,1,1)(0,0,0)[0] intercept   : AIC=-18197.666, Time=0.65 sec
 ARIMA(0,1,0)(0,0,0)[0]             : AIC=-18200.680, Time=0.13 sec

Best model:  ARIMA(0,1,0)(0,0,0)[0]          
Total fit time: 2.060 seconds
[0.8676366 0.8676366 0.8676366 0.8676366 0.8676366]


127.0.0.1 - - [05/Jun/2022 21:09:59] "OPTIONS / HTTP/1.1" 200 -


[*********************100%***********************]  1 of 1 completed
Performing stepwise search to minimize aic
 ARIMA(1,1,1)(0,0,0)[0] intercept   : AIC=-18195.765, Time=0.45 sec
 ARIMA(0,1,0)(0,0,0)[0] intercept   : AIC=-18199.610, Time=0.31 sec
 ARIMA(1,1,0)(0,0,0)[0] intercept   : AIC=-18197.667, Time=0.39 sec


127.0.0.1 - - [05/Jun/2022 21:10:02] "GET / HTTP/1.1" 200 -


 ARIMA(0,1,1)(0,0,0)[0] intercept   : AIC=-18197.666, Time=0.64 sec
 ARIMA(0,1,0)(0,0,0)[0]             : AIC=-18200.680, Time=0.11 sec

Best model:  ARIMA(0,1,0)(0,0,0)[0]          
Total fit time: 1.937 seconds
[0.8676366 0.8676366 0.8676366 0.8676366 0.8676366]
